<a href="https://colab.research.google.com/github/jcdevaney/pyAMPACTtutorials/blob/main/01-pyAMPACT_introduction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1>pyAMPACT Introduction</h1>

In [2]:
!git clone https://github.com/jcdevaney/pyAMPACTtutorials.git
from IPython.utils import io
print('Importing libraries...')
with io.capture_output() as captured:
    !pip install --upgrade pandas
    !pip install -i https://test.pypi.org/simple/ --no-deps pyampact
    import pyampact
import pandas as pd
import numpy as np
import librosa

Cloning into 'pyAMPACTtutorials'...
remote: Enumerating objects: 420, done.
remote: Counting objects: 100% (243/243), done.
remote: Compressing objects: 100% (137/137), done.
remote: Total 420 (delta 149), reused 177 (delta 105), pack-reused 177
Receiving objects: 100% (420/420), 19.44 MiB | 15.25 MiB/s, done.
Resolving deltas: 100% (210/210), done.
Importing libraries...


### Load audio with librosa and symbolic files with pyAMPACT's `Score` function. Audio files can be any format supported by librosa (wav, mp3, flac, ogg, etc.) and symbolic files can be any format supported by music21 (kern, xml, mei, midi, abc, etc.).

In [3]:
audio_file = '/content/pyAMPACTtutorials/test_files/Mozart_K179_seg.wav'
score_file = '/content/pyAMPACTtutorials/test_files/Mozart_K179_seg.krn'

y, original_sr = librosa.load(audio_file)

piece = pyampact.Score(score_file)

### Run the alignment with pyampact's `run_alignment` function.



In [4]:
# You can specify the target sample rate (target_sr), window size in
# milliseconds (win_ms), the hop size (hop_length) as well as arguments
# that dictate the way that the symbolic file is represented as a
# spectrogram-like mask: the bin width of each note/harmonic (width)
# and the number of harmonics (n_harm).
target_sr = 4000
win_ms = 100
hop_length = 32
width = 3
n_harm = 3

res, dtw, spec, nmat = pyampact.run_alignment(
    y, original_sr, piece, piece.nmats(), width, target_sr, n_harm, win_ms, hop_length)

### Estimate performance parameters

In [10]:
ons = res['on']
offs = res['off']
midi_notes = np.array(res['nmat']['Part-1']['MIDI'])

for loc in range(len(ons)):
    #Estimate f0, power, and spectrum for a matrix (or vector) of amplitudes and frequencies
    [f0, pwr, t, M, xf] = f0_est_weighted_sum_spec(audio_file, ons[loc], offs[loc], midi_notes[loc]);
    # Estimate note-wise perceptual values
    note_vals.append(estimate_perceptual_parameters(f0, pwr, M, original_sr, 256, 1))

NameError: name 'f0_est_weighted_sum_spec' is not defined

### Compile estimated performance parameters

In [ ]:
performData = pyampact.data_compilation(f0_values, sig_pwr, mags_mat, nmat, target_sr, hop_length, y, audio_file)

### Export the performance data to Kern or MEI

In [ ]:
piece.toKern()

'!!!COM: Mozart\n!!!OTL: Zwolf Variationen in C uber ein Menuett von Johann Christian Fischer\n**kern\t**kern\n*part2\t*part1\n*staff2\t*staff1\n*Ivox\t*Ivox\n*I"Part-2\t*I"Part-1\n*I\'P\t*I\'P\n=1\t=1\n*clefF4\t*clefG2\n*k[]\t*k[]\n*M3/4\t*M3/4\n4C\t2gg\n4E\t.\n4C\t16ffLL\n.\t16ee\n.\t16dd\n.\t16ccJJ\n=2\t=2\n4D\t8ggL\n.\t8ffJ\n4BB\t4.dd\n4GG\t.\n.\t8ff\n=3\t=3\n8DL\t12eeL\n.\t12gg\n8FJ\t.\n.\t12aaJ\n4G\t4cc\n4GG\t4b\n=4\t=4\n4C\t4b\n4GG\t4cc\n4CC\t4r\n==\t==\n*-\t*-\n!!!RDF**kern: %=rational rhythm\n!!!RDF**kern: l=long note in original notation\n!!!RDF**kern: i=editorial accidental\n!!!ONB: Translated from a krn file on 2024-04-09 via pyAMPACT\n!!!title: @{OTL}'

In [ ]:
piece.toMEI()